In [ ]:
# default_exp core

# Core

> Helper functions and utilities.

In [ ]:
#export
import requests, json, datetime, collections, typing, urllib
from pathlib import Path
from fastcore.basics import patch_to

In [ ]:
#export
class DotPathDict(collections.UserDict):
    "Wraps a `dict` to allow simple dot notation search of nested `dict`s"
    
    def __getitem__(self, dot_paths):
        "Allows dot search via subscript"
        for dot_path in dot_paths.split(' OR '):
            data, found = self.data, True
            path_parts = dot_path.split('.')
            for path_part_idx, path_part in enumerate(path_parts):
                if not isinstance(data, (dict, DotPathDict)):
                    path_parts = '.'.join(path_parts[:path_part_idx])
                    raise Exception(f'Expected "{path_parts}" to be a `dict` but found {type(data)} {data}')
                if not path_part in data:
                    found = False
                    break # try the next dot_path, if we have one
                data = data[path_part]
                if isinstance(data, list) and len(data) > 0:
                    found_next_path_part = False
                    if path_part_idx < len(path_parts) - 1:
                        next_path_part = path_parts[path_part_idx + 1]
                        for potential_data in data:
                            if next_path_part in potential_data:
                                data = potential_data
                                found_next_path_part = True
                                break
                    if not found_next_path_part:
                        data = data[0]
            if found:
                return DotPathDict(data) if isinstance(data, dict) else data

### What does `DotPathDict` do when it finds a `list`?

it'll search items in the list that will allow the seearch to continue ... TODO: write a better explanation

In [ ]:
test_resource = DotPathDict({
    'resource': {
        'resourceType': 'TestResource',
        'name': [
            'simple-name',
            {'name-type': 'complex', 'value': 'complex-name'}
        ],
        'meta': {
            'versionId': '1', 
            'source': '#dswfkjei2k3'
        }}})

In [ ]:
assert 'TestResource' == test_resource['resource.resourceType']
assert None is test_resource['resource.id']
assert 'TestResource' == test_resource['resource.id OR resource.resourceType']
test_resource_meta = test_resource['resource.meta']
assert '#dswfkjei2k3' == test_resource_meta['source']
# show how lookahead works
assert 'simple-name' == test_resource['resource.name']
assert 'complex-name' == test_resource['resource.name.value']

In [ ]:
#export
def request_to_cache_file(url, params):
    "Return the local file that is or should be used to cache the request"
    parsed_url = urllib.parse.urlparse(url)
    cache_folder = Path('data/cache/')/parsed_url.netloc.replace(':', '-')
    cache_index = {}
    if (cache_folder/'index.json').is_file():
        with open(cache_folder/'index.json') as f:
            cache_index = json.load(f)
    cache_key = url if params is None else f'{url}?{urllib.parse.urlencode(params)}'
    if cache_key in cache_index:
        return cache_folder/cache_index[cache_key]
    cache_index[cache_key] = f'''{parsed_url.path.replace('/', '')}-{len(cache_index)}'''
    cache_folder.mkdir(parents=True, exist_ok=True)
    with open(cache_folder/'index.json', 'w') as f:
        json.dump(cache_index, f, indent=2)
    return cache_folder/cache_index[cache_key]

In [ ]:
f = request_to_cache_file('http://ips.health:8080/fhir/Patient', dict(_id='fakeId'))
assert f.name.startswith('fhirPatient-')
assert ('data', 'cache', 'ips.health-8080') == f.parent.parts
assert Path('data/cache/ips.health-8080/index.json').is_file()

In [ ]:
f = request_to_cache_file('http://ips.health/fhir/Patient', dict(_id='fakeId'))
assert f.name.startswith('fhirPatient-')
assert ('data', 'cache', 'ips.health') == f.parent.parts
assert Path('data/cache/ips.health/index.json').is_file()

In [ ]:
#export
def request_from_github(url, params):
    "GET a cached response from github"
    netloc = urllib.parse.urlparse(url).netloc
    raw_branch = 'https://raw.githubusercontent.com/pete88b/vulcan_rwd_ig/main'
    response = requests.get(f'{raw_branch}/data/cache/{netloc}/index.json')
    print('GET', response.url, 'Status', response.status_code)
    cache_index = response.json()
    cache_key = url if params is None else f'{url}?{urllib.parse.urlencode(params)}'
    if cache_key not in cache_index:
        raise Exception(f'Query not cached in github: {cache_key} not in {response.url}')
    return requests.get(f'{raw_branch}/data/cache/{netloc}/{cache_index[cache_key]}')

In [ ]:
github_response = request_from_github('https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io/Patient', {
    'birthdate': 'le1996-09-01',
    'gender': 'male,female'
})
assert 'd1b67ca8-3de4-11ed-87ec-02f861e3f62a' == github_response.json()['id']

GET https://raw.githubusercontent.com/pete88b/vulcan_rwd_ig/main/data/cache/fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io/index.json Status 200


In [ ]:
#export
class CachedResponse:
    "Looks like an HTTP response but loads a locally cached response instead"
    def __init__(self, cache_file):
        self.status_code = 200
        with open(cache_file) as f:
            self.text = f.read()
    def json(self):
        return json.loads(self.text)

In [ ]:
r = CachedResponse('test/data/Patient_bundle.json')
assert '607d0e75-a3b9-4382-a19e-ffac65b1a521' == r.json()['id']
assert 200 == r.status_code

In [ ]:
#export
def requests_get(url, params, headers=None, use_local_cache=True):
    "Returns an HTTP response from local cache, FHIR server or github"
    if use_local_cache:
        cache_file = request_to_cache_file(url, params)
        if cache_file.is_file():
            print('GET', cache_file, 'from local cache')
            return CachedResponse(cache_file)
    # HTTP GET from a real FHIR server
    response = requests.get(url, params, headers=headers)
    print('GET', response.url, 'Status', response.status_code)
    if not response.status_code:
        # HTTP GET from github if the real server doesn't return a successful response
        response = request_from_github(url, params)
        print('GET', response.url, 'Status', response.status_code)
    # cache the result returned by the real FHIR server
    if use_local_cache and response.status_code == 200:
        cache_file.parent.mkdir(parents=True, exist_ok=True)
        with open(cache_file, 'w', encoding='utf8') as f:
            f.write(response.text)
    return response

TODO: explain ...

In [ ]:
#export
class FhirClient:
    "Helps to GET FHIR resources"
    def __init__(self, api_base:str, x_api_key:str=None, use_local_cache=True):
        self.api_base = api_base
        self.use_local_cache = use_local_cache
        self.request_headers = {}
        if x_api_key is not None:
            self.request_headers['x-api-key'] = x_api_key
        self.default_params = {}
    
    def get_as_raw_json(self, resource_type:str, params:dict=None) -> dict:
        "GET FHIR resources of `resource_type` in JSON format"
        url = f'{self.api_base}/{resource_type}'
        params = self.default_params if params is None else params
        return requests_get(url, params, self.request_headers, self.use_local_cache).json()
    
    def get_next_as_raw_json(self, json_response:dict) -> dict:
        "GET the next set of results"
        if 'link' not in json_response:
            return None
        for link in json_response['link']:
            if link['relation'] == 'next':
                url = link['url']
                response = requests_get(url, None, self.request_headers, self.use_local_cache)
                print('GET', url, 'Status', response.status_code)
                return response.json()

In [ ]:
client = FhirClient('https://ips.health/fhir')
response = client.get_as_raw_json('Patient', dict(_id='fakeId'))
assert isinstance(response, dict)

GET data\cache\ips.health\fhirPatient-1 from local cache


In [ ]:
client = FhirClient('https://ips.health/fhir', use_local_cache=False)
response = client.get_as_raw_json('Patient', dict(_id='fakeId'))
assert isinstance(response, dict)

GET https://ips.health/fhir/Patient?_id=fakeId Status 200


In [ ]:
#export
@patch_to(FhirClient)
def get_all_entries(self, resource_type:str, params:dict=None, page_limit:int=100) -> typing.List[DotPathDict]:
    "Return a list of entries of `resource_type` in JSON format while taking care of bundle pageing"
    page_count, result = 0, []
    bundle = self.get_as_raw_json(resource_type, params)
    total = bundle.get('total', 'Unknown')
    if total == 0:
        print('Returning', len(result), 'entries')
        return result
    while bundle is not None:
        if bundle.get('resourceType', None) != 'Bundle':
            raise Exception(f'Expected a bundle but found', bundle) # might be {'resourceType': 'OperationOutcome' ... 
        result.extend(bundle['entry']) # todo check for OperationOutcome etc in `entry`
        page_count += 1
        if page_count > page_limit:
            print('Stopping early. Will return', len(result), 'entries out of total', total)
            break
        bundle = self.get_next_as_raw_json(bundle)
    def _expected_resource_type(resource):
        actual_resource_type = resource.get('resource', {}).get('resourceType', None)
        if actual_resource_type != resource_type:
            print('Removing resource. Expected', resource_type, 'but found', actual_resource_type)
            return False
        return True
    result = [r for r in result if _expected_resource_type(r)]
    result = [DotPathDict(r) for r in result]
    print('Returning', len(result), 'entries')
    return result

@patch_to(FhirClient)
def get_all_resources(self, resource_type:str, params:dict=None, page_limit:int=100):
    "Return a list of resources of `resource_type` in JSON format"
    result = self.get_all_entries(resource_type, params, page_limit)
    result = [r['resource'] for r in result]
    return result

@patch_to(FhirClient)
def get_by_reference(self, reference:str):
    "Return a resource read from a FHIR server by reference, as a list containg a single bundle entry"
    if reference.startswith(self.api_base):
        reference = reference[len(self.api_base):].strip('/')
    if reference.startswith('http'):
        print(f'WARNING: Found reference {reference} that does not start with {api_base}')
        return []
    resource_type, id = reference.split('/')
    single_resource = self.get_as_raw_json(resource_type, id)
    return [dict(fullUrl = f'{self.api_base}/{resource_type}/{id}', resource = single_resource)]

In [ ]:
# TODO: remove this code when we have cached data to github
client = FhirClient('https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io')
patient_resources = client.get_all_resources('Patient', {
    'birthdate': 'le1996-09-01',
    'gender': 'male,female'
})
encounter_resources = client.get_all_resources('Encounter', {
        'reason-code': 'I20,I21,I22,I23,I24,I25', # TODO: not using below for now 'reason-code:below': 'I20,I21,I22,I23,I24,I25',
        'date': ['ge2014-09-01', 'le2015-09-30'], # TODO: Not sure this is filtering as we want ...
        'status': 'finished',
#         'dischargeDisposition:not':'exp' # TODO: Do this client side for now
})
medication_administration_resources = client.get_all_resources('MedicationAdministration', {
        'status': 'completed',
#         'effective-time': 'ge[Encounter-Start-Date]', # TODO: don't think this is possible via FHIR query
        'code': 'http://www.nlm.nih.gov/research/umls/rxnorm|1116632,'
                'http://www.nlm.nih.gov/research/umls/rxnorm|613391,'
                'http://www.nlm.nih.gov/research/umls/rxnorm|32968,'
                'http://www.nlm.nih.gov/research/umls/rxnorm|687667,'
                'http://www.nlm.nih.gov/research/umls/rxnorm|153658'})

GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\Patient-0 from local cache
Returning 29 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\Encounter-1 from local cache
Returning 63 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationAdministration-2 from local cache
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationAdministration-3 from local cache
GET https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io/MedicationAdministration?page=2&queryId=032de5e2-3de4-11ed-a7cb-02f861e3f62a Status 200
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationAdministration-4 from local cache
GET https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io/MedicationAdministration?page=3&queryId=032de5e2-3de4-11ed-a7cb-02f861e3f62a Status 200
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationAdministration-5 from local cache
GET https://fhir.ggy

In [ ]:
# TODO: remove
from uuid import uuid4
def extract_references_from_resource(resource, field_name):
    "Return a list of references extracted from a single resource and field"
    result = []
    if field_name in resource:
        references = resource[field_name]
        if not isinstance(references, list): references = [references]
        for reference in references:
            _reference = reference.get('reference')
            if _reference is None: 
                continue
            if _reference.startswith('#'): 
                continue
            # TODO: check that we have a relative reference or handle other kinds too
            result.append(_reference)
    return result

def extract_references(entries, field_names):
    "Return a list of relative references e.g. `['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8']`"
    result = []
    for entry in entries:
        resource = entry['resource']
        for f in field_names:
            result.extend(extract_references_from_resource(resource, f))
    return list(set(result)) # de-duplicate but still return a list

def timestamp_now():
    return datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')

def new_bundle(bundle_type='collection'):
    return dict(resourceType='Bundle', 
                id=str(uuid4()),
                type=bundle_type, 
                timestamp=timestamp_now(),
                entry=[])

def create_single_patient_bundle(patient_id):
    "Return a Bundle containing one Patient and ... when we don't have IPS available"
    result = new_bundle()
    references = []
    for resource_type, params in [
            ['Patient', dict(_id=patient_id)],
            ['AllergyIntolerance', dict(patient=patient_id)],
            ['Condition', dict(patient=patient_id)],
            ['Procedure', dict(patient=patient_id)],
            ['MedicationRequest', dict(subject=f'Patient/{patient_id}')],
            ['MedicationDispense', dict(subject=f'Patient/{patient_id}')],
            ['MedicationAdministration', dict(subject=f'Patient/{patient_id}')],
            ['MedicationStatement', dict(subject=f'Patient/{patient_id}')]]:
        single_resource_entries = client.get_all_entries(resource_type, params)
        result['entry'].extend(single_resource_entries)
        references.extend(extract_references(single_resource_entries, ['medicationReference', 'reasonReference']))
        
    for reference in set(references):
        try:
            result['entry'].extend(client.get_by_reference(reference))
        except Exception as ex:
            print(f'Failed to reference {reference} from {client.api_base}\n{ex}')
    return result


for patient_id in [
        'Patient/e4c9f85f8b2b9a85b32f7d9a67ea1046',
        'Patient/51ff4d27ccf78c1d2ff6438175b541c7',
        'Patient/1bbc2bc53ed277ac09507e6893743410',
        'Patient/6863ac983b0b55455da78f1fdd1288ff',
        'Patient/897dfe86f0b710793927d8034e568ee4',
        'Patient/a3f4f0ffc5c2fb1a4708452a485d1442',
        'Patient/ceba63b6dcbda783668cf3efeea1d3dd',
        'Patient/00d7dde9ae58163184c3836f01deff61',
        'Patient/f7f2e775f7ae3f7a095146cb4deaa497',
        'Patient/ff7c22942a1e16167f1b9c44f12aae05']:
    create_single_patient_bundle(patient_id)

GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\Patient-13 from local cache
Returning 0 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\AllergyIntolerance-14 from local cache
Returning 0 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\Condition-15 from local cache
Returning 26 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\Procedure-16 from local cache
Returning 0 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationRequest-17 from local cache
Returning 0 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationDispense-18 from local cache
Returning 0 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationAdministration-19 from local cache
Returning 0 entries
GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\MedicationStatement-20 from local cache
Returning 0 entries
GET data\cache

## Now we can run lots of queries (like &darr;) to load up our local cache
```
client = FhirClient('https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io')

patient_resources = client.get_all_resources('Patient', {
    'birthdate': 'le1996-09-01',
    'gender': 'male,female'
})
```
... so that subsequent calls will use cached results

In [ ]:
patient_resources = client.get_all_resources('Patient', {
    'birthdate': 'le1996-09-01',
    'gender': 'male,female'
})

GET data\cache\fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io\Patient-0 from local cache
Returning 29 entries


In [ ]:
#export
def extract_patient_ids(resources):
    "Return a list relative references of all patients found in a `resources`"
    # Note: no checks are made that the bundle contains resources of the same type etc
    result = []
    for resource in resources:
        if resource['resourceType'] == 'OperationOutcome':
            continue # e.g. "Unrecognized parameter 'dischargeDisposition'. exp"
        if resource['resourceType'] == 'Patient':
            result.append('Patient/' + resource['id'])
        else:
            result.append(resource['subject.reference OR patient.reference'])
    return result

In [ ]:
#export
def intersection_patient_ids(*resource_lists):
    "Returns a list of references for all patients found in all resource lists"
    all_patient_ids = []
    for resource_list in resource_lists:
        all_patient_ids.append(extract_patient_ids(resource_list))
    all_patient_ids = [set(ids) for ids in all_patient_ids]
    result = all_patient_ids[0]
    for ids in all_patient_ids[1:]:
        result = result & ids
    return list(result)

In [ ]:
#export
def _find_by_key(d, k, result):
    if isinstance(d, dict):
        for _k, _v in d.items():
            if _k == k:
                result.append(_v)
            _find_by_key(_v, k, result)
    elif isinstance(d, list):
        for _v in d:
            _find_by_key(_v, k, result)
    return result

def find_by_key(d:dict, k:str):
    "Return a list of values that we're keyed by `k` at any level in `d`"
    return _find_by_key(d, k, [])

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script('00_core.ipynb')

Converted 00_core.ipynb.
